In [ ]:
pip install ogb

In [ ]:
pip install torch_geometric

In [ ]:
import os.path as osp
import time
import torch
from torch_geometric.utils import to_undirected
from torch_geometric.nn import LabelPropagation
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

# -------------------------------
# Patch torch.load for PyTorch >=2.6
# -------------------------------
torch_load_old = torch.load
def torch_load_new(*args, **kwargs):
    kwargs["weights_only"] = False
    return torch_load_old(*args, **kwargs)
torch.load = torch_load_new

# -------------------------------
# Normalization helper
# -------------------------------
def normalize_adjs(edge_index, num_nodes, edge_weight=None, eps=1e-12):
    """
    Return normalized (edge_index, edge_weight) for:
    - DAD (symmetric): D^{-1/2} A D^{-1/2}
    - GCN-DAD: same but with self-loops
    - DA  (row):       D^{-1} A
    - AD  (col):       A D^{-1}
    """
    if edge_weight is None:
        edge_weight = torch.ones(edge_index.size(1), device=edge_index.device)

    row, col = edge_index
    deg = torch.zeros(num_nodes, device=edge_weight.device).scatter_add_(0, row, edge_weight)
    deg_inv = 1.0 / deg.clamp_min(eps)
    deg_inv_sqrt = deg_inv.sqrt()

    # DAD (no self-loops)
    dad_w = deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    # GCN-style DAD (with self-loops)
    gcn_ei, gcn_w = gcn_norm(
        edge_index=edge_index, edge_weight=edge_weight,
        num_nodes=num_nodes, add_self_loops=True
    )

    # DA (row-normalized)
    da_w = deg_inv[row] * edge_weight
    # AD (col-normalized)
    ad_w = deg_inv[col] * edge_weight

    return (edge_index, dad_w), (gcn_ei, gcn_w), (edge_index, da_w), (edge_index, ad_w)

# -------------------------------
# Run LP
# -------------------------------
def run_lp(edge_index, edge_weight, y, split_idx, evaluator, name=""):
    model = LabelPropagation(num_layers=1, alpha=1.0)
    out = model(y, edge_index, mask=split_idx['train'], edge_weight=edge_weight)
    y_pred = out.argmax(dim=-1, keepdim=True)

    accs = {}
    for split in ["train", "valid", "test"]:
        accs[split] = evaluator.eval({
            "y_true": y[split_idx[split]],
            "y_pred": y_pred[split_idx[split]],
        })["acc"]

    print(f"[{name}] "
          f"Train: {accs['train']:.4f} | "
          f"Val: {accs['valid']:.4f} | "
          f"Test: {accs['test']:.4f}")
    return accs

# -------------------------------
# Load dataset
# -------------------------------
root = './data/OGB'
dataset = PygNodePropPredDataset('ogbn-arxiv', root=root, transform=None)
split_idx = dataset.get_idx_split()
graph = dataset[0]
evaluator = Evaluator(name='ogbn-arxiv')

print("OGB Arxiv dataset loaded successfully!")
print(f"num_nodes: {graph.num_nodes}, num_edges: {graph.num_edges}\n")

# -------------------------------
# Helper: evaluate one graph
# -------------------------------
def evaluate_graph(name, edge_index, edge_weight, y, split_idx, evaluator, num_nodes):
    results = {}
    (DAD, GCN_DAD, DA, AD) = normalize_adjs(edge_index, num_nodes, edge_weight)

    results["DAD"]     = run_lp(*DAD,     y, split_idx, evaluator, f"{name} LP (DAD)")
    results["GCN-DAD"] = run_lp(*GCN_DAD, y, split_idx, evaluator, f"{name} LP (GCN-DAD)")
    results["DA"]      = run_lp(*DA,      y, split_idx, evaluator, f"{name} LP (DA)")
    results["AD"]      = run_lp(*AD,      y, split_idx, evaluator, f"{name} LP (AD)")
    return results

# -------------------------------
# Step 1: Original adjacency A
# -------------------------------
edge_index = to_undirected(graph.edge_index, num_nodes=graph.num_nodes)
print("=== Original Graph (A) ===")
results_A = evaluate_graph("OGB-Arxiv", edge_index, None, graph.y, split_idx, evaluator, graph.num_nodes)

# -------------------------------
# Step 2: Transformed adjacency
# -------------------------------
num_nodes, num_edges = graph.num_nodes, graph.num_edges
D = torch.sparse_coo_tensor(
    graph.edge_index,
    torch.ones(num_edges, device=graph.x.device),
    (num_nodes, num_nodes)
)

start = time.time()
D2 = torch.sparse.mm(D, D)
D3 = torch.sparse.mm(D, D2)
DMI = 2*D+D2
print(f"\nSNN(beta,(1,1)) bult in {time.time()-start:.2f}s")

w = DMI.coalesce()
edge_index_dmi, edge_weight_dmi = w.indices(), w.values()
edge_index_dmi, edge_weight_dmi = to_undirected(edge_index_dmi, edge_weight_dmi)

print("\n=== Transformed Graph (SNN(beta,(1,1))) ===")
results_DMI = evaluate_graph("DMI", edge_index_dmi, edge_weight_dmi, graph.y, split_idx, evaluator, num_nodes)

# -------------------------------
# Final summary
# -------------------------------
print("\n================= SUMMARY =================")
for graph_name, results in [("OGB-Arxiv", results_A), ("SNN(beta,(1,1))", results_DMI)]:
    print(f"\n{graph_name}:")
    for norm_name, accs in results.items():
        print(f"{norm_name:8s} -> Train: {accs['train']:.4f} | "
              f"Val: {accs['valid']:.4f} | Test: {accs['test']:.4f}")


In [ ]:
import os.path as osp
import time
import torch
from torch_geometric.utils import to_undirected
from torch_geometric.nn import LabelPropagation
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

# -------------------------------
# Patch torch.load for PyTorch >=2.6
# -------------------------------
torch_load_old = torch.load
def torch_load_new(*args, **kwargs):
    kwargs["weights_only"] = False
    return torch_load_old(*args, **kwargs)
torch.load = torch_load_new

# -------------------------------
# Normalization helper
# -------------------------------
def normalize_adjs(edge_index, num_nodes, edge_weight=None, eps=1e-12):
    """
    Return normalized (edge_index, edge_weight) for:
    - DAD (symmetric): D^{-1/2} A D^{-1/2}
    - GCN-DAD: same but with self-loops
    - DA  (row):       D^{-1} A
    - AD  (col):       A D^{-1}
    """
    if edge_weight is None:
        edge_weight = torch.ones(edge_index.size(1), device=edge_index.device)

    row, col = edge_index
    deg = torch.zeros(num_nodes, device=edge_weight.device).scatter_add_(0, row, edge_weight)
    deg_inv = 1.0 / deg.clamp_min(eps)
    deg_inv_sqrt = deg_inv.sqrt()

    # DAD (no self-loops)
    dad_w = deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    # GCN-style DAD (with self-loops)
    gcn_ei, gcn_w = gcn_norm(
        edge_index=edge_index, edge_weight=edge_weight,
        num_nodes=num_nodes, add_self_loops=True
    )

    # DA (row-normalized)
    da_w = deg_inv[row] * edge_weight
    # AD (col-normalized)
    ad_w = deg_inv[col] * edge_weight

    return (edge_index, dad_w), (gcn_ei, gcn_w), (edge_index, da_w), (edge_index, ad_w)

# -------------------------------
# Run LP
# -------------------------------
def run_lp(edge_index, edge_weight, y, split_idx, evaluator, name=""):
    model = LabelPropagation(num_layers=1, alpha=1.0)
    out = model(y, edge_index, mask=split_idx['train'], edge_weight=edge_weight)
    y_pred = out.argmax(dim=-1, keepdim=True)

    accs = {}
    for split in ["train", "valid", "test"]:
        accs[split] = evaluator.eval({
            "y_true": y[split_idx[split]],
            "y_pred": y_pred[split_idx[split]],
        })["acc"]

    print(f"[{name}] "
          f"Train: {accs['train']:.4f} | "
          f"Val: {accs['valid']:.4f} | "
          f"Test: {accs['test']:.4f}")
    return accs

# -------------------------------
# Load dataset
# -------------------------------
root = './data/OGB'
dataset = PygNodePropPredDataset('ogbn-arxiv', root=root, transform=None)
split_idx = dataset.get_idx_split()
graph = dataset[0]
evaluator = Evaluator(name='ogbn-arxiv')

print("OGB Arxiv dataset loaded successfully!")
print(f"num_nodes: {graph.num_nodes}, num_edges: {graph.num_edges}\n")

# -------------------------------
# Helper: evaluate one graph
# -------------------------------
def evaluate_graph(name, edge_index, edge_weight, y, split_idx, evaluator, num_nodes):
    results = {}
    (DAD, GCN_DAD, DA, AD) = normalize_adjs(edge_index, num_nodes, edge_weight)

    results["DAD"]     = run_lp(*DAD,     y, split_idx, evaluator, f"{name} LP (DAD)")
    results["GCN-DAD"] = run_lp(*GCN_DAD, y, split_idx, evaluator, f"{name} LP (GCN-DAD)")
    results["DA"]      = run_lp(*DA,      y, split_idx, evaluator, f"{name} LP (DA)")
    results["AD"]      = run_lp(*AD,      y, split_idx, evaluator, f"{name} LP (AD)")
    return results

# -------------------------------
# Step 1: Original adjacency A
# -------------------------------
edge_index = to_undirected(graph.edge_index, num_nodes=graph.num_nodes)
print("=== Original Graph (A) ===")
results_A = evaluate_graph("OGB-Arxiv", edge_index, None, graph.y, split_idx, evaluator, graph.num_nodes)

# -------------------------------
# Step 2: Transformed adjacency (DMI = 3D + 3D^2 + D^3)
# -------------------------------
num_nodes, num_edges = graph.num_nodes, graph.num_edges
D = torch.sparse_coo_tensor(
    graph.edge_index,
    torch.ones(num_edges, device=graph.x.device),
    (num_nodes, num_nodes)
)

start = time.time()
D2 = torch.sparse.mm(D, D)
D3 = torch.sparse.mm(D, D2)
DMI = 3*D + 3*D2 + D3
print(f"\nSNN(beta,(1,1,1)) bult in {time.time()-start:.2f}s")

w = DMI.coalesce()
edge_index_dmi, edge_weight_dmi = w.indices(), w.values()
edge_index_dmi, edge_weight_dmi = to_undirected(edge_index_dmi, edge_weight_dmi)

print("\n=== Transformed Graph (SNN(beta,(1,1,1))) ===")
results_DMI = evaluate_graph("DMI", edge_index_dmi, edge_weight_dmi, graph.y, split_idx, evaluator, num_nodes)

# -------------------------------
# Final summary
# -------------------------------
print("\n================= SUMMARY =================")
for graph_name, results in [("OGB-Arxiv", results_A), ("SNN(beta,(1,1,1))", results_DMI)]:
    print(f"\n{graph_name}:")
    for norm_name, accs in results.items():
        print(f"{norm_name:8s} -> Train: {accs['train']:.4f} | "
              f"Val: {accs['valid']:.4f} | Test: {accs['test']:.4f}")
